#### 0. saspy

In [1]:
import saspy
from IPython.display import HTML

#### 1. Config

In [2]:
sascfg="c:/code/pub/ct/sascfg.py"
sas=saspy.SASsession(cfgfile=sascfg)
sas.HTML_Style="PowerPointDark"
_=sas.submit("""
%let ROOT=/home/u63544628;
libname CUBE "/home/u63544628/cubedemo";
""")

Using SAS Config named: oda
SAS Connection established. Subprocess id is 10288



#### 2. BL - Manipulating

In [3]:
resp=sas.submit("""
title font="Consolas" "BL";

proc import datafile="&ROOT./bl.csv" out=bl dbms=csv replace;
    getnames=yes; guessingrows=300;
    run;

data bl_; set bl;
    rename RESULT=SCORE;
    format TYPE $3. RESULT 1. TYPEX $10.;
    INPUT=trim(OUTPUT);
    OUTPUT=trim(OUTPUT);
    RESULT=put(substr(RESULT,1,1),1.);

    NAN=cmiss(of BATCH--RESULT);

    if BATCH="MAS" then TYPE="MAC";
    else if BATCH="AAS" then TYPE=.;
    else TYPE="#NA";

    TYPEX=put(cats(BATCH,"-",LOCALE,"-",TYPE),$10.);

    SINCE=intck("day",DATE,today());

    if cmiss(of _all_) then delete;

    run;

data bl_; set bl_; keep BATCH INPUT OUTPUT SCORE DATE NAN SINCE; run;

proc sort data=bl_ out=bl_; by INPUT; run;
proc sort data=bl out=bl; by INPUT; run;

data bl_; merge bl_(in=x) bl(in=y); by INPUT; if x; run;

data bl_; set bl_; drop BATCH; IDX="bl"||left(put(_N_,10.)); run;

proc print data=bl_(obs=3) noobs; run;
""")
HTML(resp["LST"])

INPUT,OUTPUT,DATE,SCORE,NAN,SINCE,LOCALE,RESULT,IDX
#1 Companion For Fortnit,#1 Companion For Fortnite,2023-04-15,4,0,157,,,bl1
#DRIVE,#DRIVE,2023-05-12,2,0,130,,,bl2
#DRIVE,#DRIVE,2022-07-18,5,0,428,,,bl3


#### 3. CUBE - CMDM - Manipulating

In [8]:
resp=sas.submit("""
title font="Consolas" "DMCM with CMINDCAE";

/* 포맷 딕셔너리 구성 */
proc format;
	value BOOL .="False" 1="True";
	value YN .="none" 1="Yes" 2="No";
	value SEX .="none" 1="Nam" 2="Yeo";
	run;

/* DM 딕셔너리 구성, SUBJID 축약 */
data DM; set CUBE.DM;
    keep SUBJID AGE SEX IDXDM;
	SUBJID=compress(SUBJID,"-");
    IDXDM="DM"||left(_N_);
	run;

/* CM 정리 */
data CM; set CUBE.CM;
    format CMONGO BOOL. CMSTATUS $10.;
    where not missing(CMINDCAE);
    
    SUBJID=compress(SUBJID,"-");
    IDXCM="CM"||left(_N_);
    CMINDCAE=substr(CMINDCAE,find(CMINDCAE,"^","i")+1);
    
    if CMONGO=. then CMSTATUS="none";
    else if CMONGO=1 then CMSTATUS="ongoing";
    else CMSTATUS="";
    
    select (CMONGO);
        when (.) CMSTAT=".";
        when ("0") CMSTAT="000";
        when ("1") CMSTAT="111";
        otherwise CMSTAT="_";
        end;

    array varchar{*} _character_;
        do q=1 to dim(varchar);
        varchar{q}=trim(upcase(varchar{q}));
        end;
    run;

/* CM_에 DM 딕셔너리 left join */
data CM_; retain SUBJID SEX AGE CMTRT CMINDCAE CMSTAT CMSTATUS IDXDM IDXCM; set CM;
    keep SUBJID SEX AGE CMTRT CMINDCAE CMSTAT CMSTATUS IDXDM IDXCM;
    format SEX SEX.;
    merge CM(in=x) DM(in=y); by SUBJID; if x;
    run;

/* sort_values(by="SUBJID") */
proc sort data=CM_ out=CM_; by SUBJID CMTRT; run;

proc print data=CM_(obs=5) noobs; run;
""")
HTML(resp["LST"])

SUBJID,SEX,AGE,CMTRT,CMINDCAE,CMSTAT,CMSTATUS,IDXDM,IDXCM
S1Z018,Yeo,39,COUGH SYR 20ML,PNEUMONIA,.,NONE,DM11,CM2
S1Z018,Yeo,39,PLAKON POWD 3MG,LOCALIZED ITCHING,.,NONE,DM11,CM1
S1Z020,Nam,48,COUGH SYR 20ML,COMMON COLD,.,NONE,DM13,CM3
S1Z022,Yeo,38,DOPAMINE DAEWOO INJ 200MG/5ML,ANXIETY AGGRAVATED,.,NONE,DM15,CM5
S1Z022,Yeo,38,XANAX XR TAB 0.5MG,ANXIETY AGGRAVATED,.,NONE,DM15,CM4


#### 4. CUBE - CMDMLB - Manipulating

In [5]:
resp=sas.submit("""
title font="Consolas" "LB w/o LBTEST";

data LB; retain SUBJID LBVISIT LBSEQ; set CUBE.LB;
    keep SUBJID LBVISIT LBSEQ LBTEST LBORRES IDXLB;
	SUBJID=compress(SUBJID,"-");
    IDXLB="LB"||left(_N_);
	LBTEST=trim(upcase(LBTEST));
    LBVISIT=VISIT;
    LBSEQ=SEQ;
	run;

proc print data=LB(where=(LBTEST="")) noobs; run;
""")
HTML(resp["LST"])

SUBJID,LBVISIT,LBSEQ,LBTEST,LBORRES,IDXLB
SHW001,1,14,,9.68,LB2859


In [6]:
resp=sas.submit("""
title font="Consolas" "Unstacking LB";

proc sort data=LB out=LB_;
	by SUBJID LBVISIT LBSEQ;
	run;

proc transpose data=LB_ out=LB_(drop=_NAME_);
	by SUBJID LBVISIT LBSEQ; id LBTEST; var LBORRES;
	run;

proc print data=LB_(obs=5) noobs; run;
""")
HTML(resp["LST"])

SUBJID,LBVISIT,LBSEQ,ERYTHROCYTES,HEMOGLOBIN,HEMATOCRIT,PLATELETS,LEUKOCYTES,PROTEIN,ALBUMIN,ASPARTATE AMINOTRANSFERASE,ALANINE AMINOTRANSFERASE,BILIRUBIN,HEMOGLOBIN A1C,SPECIFIC GRAVITY[U],PH[U],ALBUMIN [U],OCCULT BLOOD[U],PROTEIN[U],CREATININE,"GLOMERULAR FILTRATION RATE, ESTI"
S1Z005,1,1,10,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
S1Z005,1,2,.,18,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
S1Z005,1,3,.,.,50,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
S1Z005,1,4,.,.,.,500,.,.,.,.,.,.,.,.,.,.,.,.,.,.
S1Z005,1,5,.,.,.,.,12,.,.,.,.,.,.,.,.,.,.,.,.,.


In [7]:
ctnt="""
title font="Consolas" "AESTDTC Count by Month";
data AE; retain SUBJID AETERM AESTDTC; set CUBE.AE;
	where not missing(AESTDTC);
	AESTDTC=substr(AESTDTC,1,7);
proc sort data=AE out=AE; by SUBJID AESTDTC;
proc freq data=AE noprint; table AESTDTC/out=AESTDTC(keep=AESTDTC COUNT);
proc print data=AESTDTC(obs=5) noobs;
run;
"""
resp=sas.submit(ctnt)
HTML(resp["LST"])

AESTDTC,COUNT
2004-12,1
2009-02,1
2015-02,6
2015-04,4
2016-02,16
